In [ ]:
!pip install "paho-mqtt<2.0.0"

In [ ]:
#本机测试成功，coalb第一次运行没问题，从第二次运行开始出现反复多次连接broker
import paho.mqtt.client as mqtt
import random
import time
import threading

broker_address = "broker.emqx.io"
port = 1883
username = "test"
password = "1"

# 全局字典用于跟踪每个客户端的连接状态
client_connections = {}

# 定义发布心跳数据的函数
def publish_heartbeat(client_name, topic):
    client_connected = threading.Event()

    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print(f"{client_name} connected to broker")
            client_connections[client_name] = True  # 更新连接状态
            client_connected.set()  # 设置标志表示已连接
        else:
            print(f"{client_name} connection failed with code {rc}")

    def on_disconnect(client, userdata, rc):
        print(f"{client_name} disconnected from broker with code {rc}")
        client_connections[client_name] = False  # 更新连接状态
        client_connected.clear()  # 清除标志表示已断开连接

    if client_connections.get(client_name):
        print(f"{client_name} already connected, skipping...")
        return

    client = mqtt.Client(client_id=f"{client_name}_{int(time.time())}")
    client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.on_disconnect = on_disconnect

    client.connect(broker_address, port)
    client.loop_start()

    # 等待客户端连接成功
    if not client_connected.wait(timeout=10):
        print(f"{client_name} failed to connect within timeout")
        return

    while True:
        heartbeat = random.randint(60, 130)
        client.publish(topic, heartbeat)
        print(f"{client_name} sent heartbeat: {heartbeat}")
        time.sleep(5)

# 创建多个线程，每个线程负责一个客户端
threads = []

client_info = [
    {"client_name": "client1", "topic": "patient/num1/heartbeat"},
    {"client_name": "client2", "topic": "patient/num2/heartbeat"},
    {"client_name": "client3", "topic": "patient/num3/heartbeat"},
]

# 初始化连接状态
for info in client_info:
    client_connections[info["client_name"]] = False

for info in client_info:
    thread = threading.Thread(target=publish_heartbeat, args=(info["client_name"], info["topic"]))
    threads.append(thread)
    thread.start()

# 保持主线程运行
try:
    for thread in threads:
        thread.join()
except KeyboardInterrupt:
    print("Stopping all clients...")
    for info in client_info:
        client = mqtt.Client(client_id=f"{info['client_name']}_{int(time.time())}")
        client.disconnect()
    print("All clients stopped.")
